# Query and Accessing Data in a Gen3 Data Commons
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

June 2022





### Introduction
---
* This notebook is intended to demonstrate a variety of ways to access file objects and structured data (aka "metadata") in a Gen3 data commons.
* File objects are accessed via their "data GUID" aka "object_id", which is a unique identifier that is associated with a storage_url in the file index (https://data.midrc.org/index/index). Users must be authorized to access a file in order to download it via the object_id. 
* Structured data in a Gen3 Data Commons is imported into Postgres via the "sheepdog" service and must conform to the data model. The data model is a relational model that consists of tables or "nodes" that are related to one another via foreign keys so that the model can be thought of as a graph of nodes that are linked to each other. Each node in the model contains certain properties (keys) that store data of a particular type (values).
* The "sheepdog" service can export tables of data from a particular node of a data project. This is the simplest way to access "all" the data in a Gen3 data commons.
* Queries can be constructed to target specific types of data in Postgres and are handled by the "peregrine" graphQL service.
* Structured data can also be transformed via an "ETL" (extract, transform, load) process that takes the complex relationships between nodes and "flattens" the data into a single table, which is stored in an ElasticSearch (ES) database that can be queried using the "guppy" graphQL service. These ES tables are what the data exploration app of the Gen3 data-portal is based on.



In [ ]:
# The packages below may be necessary for users to install according to the imports necessary in the subsequent cells.

#!pip install --upgrade pandas
#!pip install --upgrade --ignore-installed PyYAML
#!pip install --upgrade pip
#!pip install --upgrade gen3
#!pip install pydicom

In [ ]:
# Import Python Packages and scripts

import pandas as pd
import sys, os
import gen3
import pydicom
from io import StringIO


from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


In [ ]:
# Import some custom Python scripts from personal GitHub repo
# change these directory paths to reflect your local working directory

home_dir = "/Users/christopher" 
demo_dir = "{}/Documents/Notes/MIDRC/tutorials".format(home_dir)

os.chdir(demo_dir)

os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O {}/expansion.py".format(demo_dir))
%run expansion.py


In [ ]:
# Initiate instances of the Gen3 SDK Classes using credentials file for authentication
# Change the directory path in "cred" to reflect the location of your credentials file.

api = "https://data.midrc.org"
cred = "{}/Downloads/midrc-credentials.json".format(home_dir)
auth = Gen3Auth(api, refresh_file=cred) # authentication class
sub = Gen3Submission(api, auth) # submission class
query = Gen3Query(auth) # query class
exp = Gen3Expansion(api,auth,sub) # class with some custom scripts
exp.get_project_ids()


## Accessing structured data in Postgres using sheepdog exports
---
* Probably the most straight-forward way to access structured data in a Gen3 Data Commons is to simply export the table of data using the sheepdog service (https://petstore.swagger.io/?url=https://raw.githubusercontent.com/uc-cdis/sheepdog/master/openapi/swagger.yml#/export/post__program___project__export).
* The Gen3SDK has a function `Gen3Submission.export_node()` for exporting entire tables of data from Postgres: https://github.com/uc-cdis/gen3sdk-python/blob/8196cf4b76a65d0b9b31c8637a18dfac2a911b56/gen3/submission.py#L361
    * This function will export all records in a particular node of a specified project, and one can then use standard Python / R (etc.) tools to do the filtering and cohort building.
* Note: This export function is also accesible in the data-portal by navigating to a data project's URL, e.g., https://data.midrc.org/Open-A1, clicking a node in the graph, and then clicking the "Download All" button.
    * For example: https://data.midrc.org/Open-A1/search?node_type=measurement
    * Or, you can enter this URL in your browser, for example: https://data.midrc.org/api/v0/submission//Open/A1/export?node_label=measurement&format=tsv

In [ ]:
## Example of exporting a table of data using the `Gen3Submission.export_node()` function
cases = sub.export_node(program='Open',project='A1',node_type='case',fileformat='tsv')
df = pd.read_csv(StringIO(cases), sep='\t', header=0)
display(df)

In [ ]:
## One can then use standard tools in any programming language to do cohort building. 
## Here I'm using the "pandas" Python package to select a cohort based on demographic information stored in the case node.
cohort = list(df.loc[(df['sex']=='Female') & (df['race']=="Black or African American") & (df['age_at_index']>79)]['submitter_id'])
display(len(cohort))
cohort

### Using a Python wrapper to get all the data in a particular node
---
* I've written a wrapper script called `Gen3Expansion.get_node_tsvs()` that uses the `Gen3Submission.export_node()` function to export the same node across all projects you have access to in the data commons and then merges the results into a single master table for that node:
https://github.com/cgmeyer/gen3sdk-python/blob/5fd6b868374f622221c0c0173a0d9489b190facd/expansion/expansion.py#L219

In [ ]:
cases = exp.get_node_tsvs(node='case')
display(cases)

### Using a Python wrapper to get all the data in a particular project
---
* Similar to the above example, I've written a wrapper script called `Gen3Expansion.get_project_tsvs()` that uses the `Gen3Submission.export_node()` function to export every node in every project (or a particular project) in the data commons.
https://github.com/cgmeyer/gen3sdk-python/blob/5fd6b868374f622221c0c0173a0d9489b190facd/expansion/expansion.py#L298
    

In [ ]:
## This example gets all the data in every node of the data model in the project Open-A1
## If "projects" is not specific, all data across all projects you have access to will be downloaded.
exp.get_project_tsvs(projects='Open-A1')

In [ ]:
!ls -l project_tsvs/Open-A1_tsvs

In [ ]:
## You can then read in the TSVs of data exported from a node to do cohort building / research
tsv_dir = 'project_tsvs/Open-A1_tsvs'
ct = pd.read_csv("{}/Open-A1_ct_series_file.tsv".format(tsv_dir),sep='\t',dtype=str)
display(ct)

In [ ]:
## Now we can use Python to get the CT series files for the cohort of cases we built earlier
cohort_ct = ct.loc[ct['case_ids'].isin(cohort)]
cohort_ct

In [ ]:
## You can access the cohort's CT series files by using the 'object_id' field:
object_ids = list(cohort_ct['object_id'])
object_ids

## Queries to Postgres using Peregrine graphQL query service
---
* Peregrine GitHub Docs: https://github.com/uc-cdis/peregrine
* Peregrine swagger docs: https://petstore.swagger.io/?url=https://raw.githubusercontent.com/uc-cdis/peregrine/master/openapis/swagger.yaml

---
* Most structured data (aka "metadata") submitted to a Gen3 system is stored in Postgres tables using the "sheepdog" service. This data must conform to the data commons' data model (https://data.midrc.org/dd), and is queryable via the "peregrine" service, which converts graphQL queries to SQL queries and returns the data requested. The Postgres tables are considered the "source-of-truth" for data in a Gen3 system (vs. the derived data in ElasticSearch, covered below).

* On the data commons' website, peregrine queries can be sent to the API using the "graphiQL" query builder: https://data.midrc.org/query (click on "Switch to Graph Model"; if button says "Switch to Flat Model" you're in the correct spot).

* Alternatively, you can send queries to the peregrine API using the Gen3SDK `Gen3Submission.query()` function, which uses the Python `requests` package to send queries as API requests: https://github.com/uc-cdis/gen3sdk-python/blob/31751633ba621b35f39eda7295f131245fb92728/gen3/submission.py#L399


### Example graph model query \#1
* This query is running across all records in the `case` node and returns data from any dataset in the data commons you are authorized to access.  Remember, the properties in the `case` node are essentially table headers for variables whose values are of a specific data type (string, enumeration, integer, number, boolean, array, etc.).
* The argument `covid19_positive: "Yes"` returns only case records that have the value "Yes" for the property `covid19_positive`, which indicates whether a case in MIDRC has ever had a positive COVID-19 test result.
* The `first` argument defines how many `case` records we want returned. Using the argument `first: 0`, all the records we have access to will be returned. If we leave the "first" argument out, only the first 10 records are returned by default. Setting `first: 2000` will return the first 2000 records in the table, etc.
* If your query is timing out, you will need to paginate the query (covered in next section) using a combination of "first" and "offset" arguments. This is only necessary if the tables being queries are very large, or the query traverses many nodes in the graph model.
* Properties we want returned from the API are enclosed in brackets. The possibilities and exact syntax are constrained by the data model (data.midrc.org/dd). 


In [ ]:
### Define the query

## Here we're asking for the `project_id`, `submitter_id`, and some demographic data for every `case` record.
## We're also asking for the `study_uid` for every `imaging_study` record belonging to those cases, and for all `dx_series_file` records for those `imaging_studies`.
## Finally, we're asking for the `file_name` and `object_id` of any Digital X-ray files (node `dx_series_file`, backref: `dx_series_files`) they may have.

## Note: "submitter_id" is a required property on every node, which is the human-readable (string), unique identifier for a record in a data table / node. So, the "submitter_id" of a record in the case node is the de-identified patient's "ID".

query_txt = """
{
  case(first: 0, covid19_positive: "Yes") {
    project_id
    submitter_id
    ethnicity
    sex
    race
    imaging_studies (study_modality: "DX") {
      study_uid
      dx_series_files {
        object_id
        file_name
      }
    }
  }
}

"""


In [ ]:
## Send the query using the Gen3 SDK Gen3Submission.query() function
## The response will be in JSON format.

response = sub.query(query_txt)
if 'data' in response:
    data = response['data']['case']
    display(data)

In [ ]:
## the "object_id" field is the file's data GUID (or globally unique identifier), which can be used to access the file.

object_ids = []
for case in data:
    studies = case['imaging_studies']
    for study in studies:
        files = study['dx_series_files'] 
        if len(files)>0:
            for file in files:
                object_id = file['object_id']
                object_ids.append(object_id)
object_ids

In [ ]:
## Take a look at one of the file objects
#object_id was originally selected from the list above.  If the object id is not above use an object id from above.

object_id = 'dg.MD1R/ea6ad8e7-1dc9-4916-8e75-38abb66c6416'
os.system("gen3 --auth {} --endpoint data.midrc.org drs-pull object {}".format(cred,object_id))

In [ ]:
!ls -l 10000364-1958844/

In [ ]:
#The object downloaded above was zipped and must be unzipped to process further
#The path and file names may have changed and those changes should be reflected below
!unzip 10000364-1958844/2.16.840.1.114274.1818.52236113359126249589212595743121753735/2.16.840.1.114274.1818.54309100269617797736626917868992258958.zip

In [ ]:
from pydicom import dcmread

fpath = "2.16.840.1.114274.1818.54309100269617797736626917868992258958/2.16.840.1.114274.1818.46312267929568121457864041736105067915.dcm"
ds = dcmread(fpath)
ds

### Counts with peregrine
---
* Peregrine is able to provide counts of records in nodes. A simple example is to quickly get the count of the numbers of cases and imaging studies in the data commons.
* You can also add arguments to the counts to, for example, get the number of cases in a particular project or get the imaging studies of a particular modality.

In [ ]:
query_txt = "{_case_count}"
print(sub.query(query_txt))
query_txt = "{_imaging_study_count}"
print(sub.query(query_txt))
query_txt = '{CT_studies: _imaging_study_count(study_modality:"CT")}'
print(sub.query(query_txt))
query_txt = '{Open_A1_cases: _case_count(project_id:"Open-A1")}'
print(sub.query(query_txt))

### Queries of "datanode" using peregrine
---
Another handy trick with peregrine queries is the "datanode" query. "Datanode" isn't a real node in the data model, but is useful way to query all nodes that store file information. For example, if you have a patient ID, you can get all the files associated with that case.


In [ ]:
query_txt = """
{
  datanode(first: 0, case_ids: "10000364-1163342") {
    object_id
    file_name
    modality
  }
}

"""
response = sub.query(query_txt)
if 'data' in response:
    display(response['data']['datanode'])

## Queries to ElasticSearch using Guppy graphQL query service
---
* Guppy Documentation: https://github.com/uc-cdis/guppy/blob/master/doc/queries.md#filters
* Guppy Download instructions: https://github.com/uc-cdis/guppy/blob/master/doc/download.md
* ETL (Tube) Documentation: https://github.com/uc-cdis/tube#gen3-etl---a-process-from-postgresql-to-es
---
* The Gen3 platform includes services for running an ETL process (Extract, Transform, Load), which is done by the Gen3 ETL service "tube", on the data in Postgres to create flattened tables of the same data in ElasticSearch (ES) for rapid querying performed by the Gen3 query service "guppy".
* Guppy runs graphql-like queries against the ES database, and can rapidly return derived data like histograms, statistics, aggregations, counts, etc. The tube service uses Spark to create these new tables of data in ES via an ETL mapping, which defines the structure of the new tables and is based on the data model. 
* Since the structure of the data changes via the ETL process, peregrine queries to Postgres will not run using guppy. To explore what is possible to query, use the graphiQL interface / documentation.
* The "Exploration" app aka "Data Explorer" (data.midrc.org/explore), which uses faceted search to filter the flat data tables in ES, runs off of guppy queries.


In [ ]:
## an example guppy query, which hits the ElasticSearch database

## define some parameters
pid = 'Open-R1'
node = 'imaging_study'
fields = ["study_uid",
    "study_description",
    "case_ids",
    "object_id"]
filters = {"project_id": pid,
    "covid19_positive" : "Yes",
    "body_part_examined" : "CHEST",
    "study_modality" : "DX"}

## send the guppy query with the SDK class Gen3Query
## Note the "first: 100000", which makes sure we don't just get the default first 10 records
response = query.query(
                data_type=node,
                first=100000,
                fields=fields,
                filters=filters,
                sort_object={"submitter_id": "asc"},
)

# display the returned data
if 'data' in response:
    study_data = response['data'][node]
    display(study_data)

In [ ]:
## another example guppy query, which hits the ElasticSearch database

## define some parameters
node = 'case'

fields = ["project_id",
    "submitter_id",
    "object_id"]

filters = {"sex":"Female",
    "race" : "Asian",
    "ethnicity" : "Hispanic or Latino"}

## send the guppy query with the SDK class Gen3Query
## Note the "first: 100000", which makes sure we don't just get the default first 10 records
response = query.query(
                data_type=node,
                first=100000,
                fields=fields,
                filters=filters,
                sort_object={"submitter_id": "asc"},
)

# display the returned data
if 'data' in response:
    case_data = response['data'][node]
    display(case_data)

In [ ]:
## Elastic search is handy for accessing files for a cohort since object_ids associated with each study or case are joined to the table 
study_object_ids = []
for study in study_data:
    if 'object_id' in study:
        object_id_list = study['object_id']
        for object_id in object_id_list:
            study_object_ids.append(object_id)
display(study_object_ids)

In [ ]:
case_data

### Sending aggregations with guppy
---
* Guppy has the ability to return some useful statistics (e.g., histograms) using aggregations.
* The `Gen3Query.graphql_query()` function can be used to send aggregations and other more complex queries that the basic `Gen3Query.query()` function can't support: https://github.com/uc-cdis/gen3sdk-python/blob/8196cf4b76a65d0b9b31c8637a18dfac2a911b56/gen3/query.py#L112

In [ ]:
## A more complex example using Python requests
query_txt = """{
    _aggregation {
        case {
          sex {
            histogram {
              key
              count
            }
          }
          race {
            histogram {
              key
              count
            }
          }
          ethnicity {
            histogram {
              key
              count
            }
          }
        }
    }
}"""
response = query.graphql_query(query_string=query_txt)
display(response)

In [ ]:
## Here is an example simple script for sending a basic aggregation request that will return the data as a DataFrame ("TSV")
## https://github.com/cgmeyer/gen3sdk-python/blob/5fd6b868374f622221c0c0173a0d9489b190facd/expansion/expansion.py#L3511

data = exp.guppy_aggregation(node='case', prop='race', format='TSV')
display(data)

In [ ]:
## A more complex example using Python requests
query_txt = """{
    _aggregation {
        case {
          sex {
            histogram {
              key
              count
            }
          }
          race {
            histogram {
              key
              count
            }
          }
          ethnicity {
            histogram {
              key
              count
            }
          }
        }
    }
}"""
query_json = {"query": query_txt}
guppy_url = "{}/guppy/graphql".format(api)
response = requests.post(guppy_url, json=query_json, auth=auth)
display(json.loads(response.text)['data']['_aggregation']['case'])


In [ ]:
## Count the number of files in each project
files_by_project = """
{
  _aggregation {
    data_file {
      project_id {
        histogram {
          key
          count
        }
      }
    }
  }
}"""
response = query.graphql_query(files_by_project)
display(response)



### Use the guppy download endpoint to access ElasticSearch tables.
---
* Tables of data from ES can be exported from the data exploration app (https://data.midrc.org/explore) by using the "Download Table" button.
* To get these sorts of tables using the API, you can use the guppy download function: https://github.com/uc-cdis/gen3sdk-python/blob/8196cf4b76a65d0b9b31c8637a18dfac2a911b56/gen3/query.py#L146

In [ ]:
## This query gets all the imaging studies of modality "CT"

query.raw_data_download(
                    data_type="imaging_study",
                    fields=[
                        "study_uid",
                        "project_id",
                        "study_description",
                        "body_part_examined",
                        "case_ids",
                        "object_id"
                    ],
                    filter_object={"=": {"study_modality": "CT"}}
                )

In [ ]:
## Here is an example getting all the cases in a particular project between ages of 45 and 47

query.raw_data_download(
                    data_type="case",
                    fields=[
                        "submitter_id",
                        "project_id",
                        "race",
                        "sex",
                        "ethnicity",
                        "age_at_index",
                        "object_id"
                    ],
                    filter_object={"AND": [{">=": {"age_at_index": 45}},
                                           {"<=": {"age_at_index": 47}},
                                           {"=": {"project_id": "Open-A1"}}]},
                    sort_fields=[{"submitter_id": "asc"}],
                    accessibility="accessible"
                )



In [ ]:
## Here is an example getting all the imaging studies where the patient had a positive COVID-19 test result within a week of the study date.

response = query.raw_data_download(
                    data_type="imaging_study",
                    fields=[
                        "study_uid",
                        "study_modality",
                        "case_ids",
                        "project_id",
                        "race",
                        "sex",
                        "ethnicity",
                        "age_at_index",
                        "object_id"
                    ],
                    filter_object={"AND": [{">=": {"days_from_study_to_pos_covid_test": -7}},
                                           {"<=": {"days_from_study_to_pos_covid_test": 7}}]},
                    sort_fields=[{"submitter_id": "asc"}],
                    accessibility="accessible"
                )

display(response)


## Use the Gen3 SDK "drs-pull" commands to access the files themselves.
---

Next, we'll use the Gen3 SDK command `gen3 drs-pull object` to access the imaging file using it's "object_id" aka "data GUID".

See the detailed documentation to learn more about the Gen3 SDK drs-pull command: https://github.com/uc-cdis/gen3sdk-python/blob/master/docs/howto/drsDownloading.md


In [ ]:
## Take a look at one of the file objects

data = response[0]
case_id = data['case_ids'][0]
study_uid = data['study_uid']
object_id = data['object_id'][0]
print(case_id)
print(object_id)

cmd = "gen3 --auth {} --endpoint data.midrc.org drs-pull object {}".format(cred,object_id)
os.system(cmd)


In [ ]:
cmd = "ls -l {}/{}".format(case_id,study_uid)
stout = subprocess.check_output(cmd, shell=True)


In [ ]:
## Grab the filename and series UID of the downloaded file using RegEx
import re

m = re.search(' ([0-9\.]+.zip)', str(stout))

if m:
    zip_file = m.group(1)
    print(zip_file)
else:
    print("No zip found.")

series_uid = re.sub("(\.zip)", "", zip_file)
print(series_uid)

In [ ]:
# Unzip the imaging series package
from zipfile import ZipFile

with ZipFile('{}/{}/{}/{}'.format(demo_dir,case_id,study_uid,zip_file), 'r') as zipObj:
    zipObj.extractall()

In [ ]:
#Input the name of the newly create .dcm file
cmd = "ls -l {}/{}/{}".format(case_id,study_uid,series_uid)
stout = subprocess.run(cmd, shell=True, capture_output=True)


In [ ]:
# Get the name of the first DICOM file in the extracted imaging series
m = re.search(' ([0-9\.]+.dcm)', str(stout))

if m:
    dcm_file = m.group(1)
    print(dcm_file)
else:
    print("No DCM files found.")


In [ ]:
# Read in the DCM file using the python DICOM package pydicom
dimg = pydicom.dcmread("{}/{}/{}/{}".format(case_id,study_uid,series_uid,dcm_file),force=True)
dimg